In [ ]:
%pip install langchain
%pip install openai

In [3]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')
os.chdir('/Users/tcoan/git_repos/ncrm-spring-school/')

In [4]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

In [107]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [12]:
llm = OpenAI(temperature=0)

In [12]:
text = """
Here is an example of a movie review:

The big Lebowski is the best movie ever! I've just stopped laughing.

Is this a positive, negative, or neutral review?" If you don't know, say 'unclear'.
"""

res = llm(text)
print(res)


Positive


In [19]:
sentiment_template = """
Here is an example of a movie review:

{review}

Is this a positive, negative, or neutral review?" If you don't know, say 'unclear'.
"""

prompt = PromptTemplate(
    input_variables=["review"],
    template=sentiment_template,
)

In [ ]:
movie_review = 'I know that most people love the movie Titanic. I thought it was pretty stupid. Sappy!'
print(prompt.format(review=movie_review))

In [22]:
print(llm(prompt.format(review=movie_review)))


Negative


In [23]:
sentiment_template = """
Here is an example of a movie review:

{review}

Is this a positive, negative, or neutral review?" If you don't know, say 'unclear'. \
Return the result as a Python dictionary with the key 'sentiment'.
"""

prompt = PromptTemplate(
    input_variables=["review"],
    template=sentiment_template,
)

In [24]:
print(llm(prompt.format(review=movie_review)))


{'sentiment': 'negative'}


In [7]:
sentiment_template = """
Here is an example of a movie review:

{review}

Is this a positive, negative, or neutral review?" If you don't know, say 'unclear'. \
Return the result as a Python dictionary with the key 'sentiment' and return a short \
description of why you gave this answer to the same dictionary using the key "reason". 
"""

prompt = PromptTemplate(
    input_variables=["review"],
    template=sentiment_template,
)

In [10]:
llm = OpenAI(temperature=0)

In [11]:
print(llm(prompt.format(review=movie_review)))


Answer: {'sentiment': 'negative', 'reason': 'The reviewer expresses dislike for the movie.'}


In [15]:
reviews = pd.read_csv('data/movie_reviews.csv').to_dict('records')

In [19]:
reviews_unlabeled = reviews[1500:]
print(len(reviews_unlabeled))

500


In [20]:
results = []
for row in reviews_unlabeled:
    results.append(llm(prompt.format(review=row['text'])))

In [108]:
positive = []
for result in results:
    if 'positive' in result[0:25]:
        positive.append(1)
    else:
        positive.append(0)


In [111]:
y = np.array([row['positive'] for row in reviews_unlabeled])
y_pred = np.array(positive)

In [112]:
accuracy_score(y, y_pred)

0.868